# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [31]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
# !pip install gmaps
!jupyter nbextension enable --py --sys-prefix gmaps
!jupyter nbextension enable --py gmaps

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
import sys
%matplotlib inline

# Import API key

# with open('/Users/tmlun/Downloads/api_keys.py', 'r') as g_key:
#     mykey=g_key.read()
#     mykey=mykey.split('=')
#     mykey=mykey[1].strip()   
# g_key = mykey.strip('"') 
from api_keys import g_key
#from api_keys import weather_api_key


# Data File path (CSV)
data = "./output_data/cities.csv"

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok
Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [32]:
Vacation_City_df = pd.read_csv(data)

Vacation_City_df.head()

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country
0,0,Jamestown,42.10,-79.24,43.00,51,90,10.29,US
1,1,Esperance,-33.87,121.90,66.20,52,40,18.34,AU
2,2,Ushuaia,-54.80,-68.30,41.00,80,75,16.11,AR
3,3,Arraial do Cabo,-22.97,-42.03,72.84,93,95,1.86,BR
4,4,Hermanus,-34.42,19.23,59.00,84,96,3.20,ZA


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [33]:

locations = Vacation_City_df[["Latitude", "Longitude"]].astype(float)
Humidity = Vacation_City_df["Humidity"].astype(float)

In [34]:
gmaps.configure(api_key=g_key)

fig= gmaps.figure(center=[0,0], zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, dissipating=False, max_intensity=100,
                                point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [35]:
Ideal_criteria_df = Vacation_City_df.loc[(Vacation_City_df["Temperature"]<95) & (Vacation_City_df["Temperature"]>75)
                                          & (Vacation_City_df["Wind Speed"]<15) & (Vacation_City_df["Wind Speed"]>5)
                                          & (Vacation_City_df["Cloudiness"]==0)].dropna()
Ideal_criteria_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Country
28,28,Mackay,-21.15,149.20,89.01,62,0,14.99,AU
59,59,Ankazoabo,-22.28,44.52,85.08,36,0,9.26,MG
66,66,Ingrāj Bāzār,25.00,88.15,85.21,35,0,7.90,IN
180,180,Shima,24.45,117.81,78.80,54,0,6.71,CN
222,222,Kareli,22.92,79.07,87.28,38,0,7.38,IN
335,335,Barão de Melgaço,-16.19,-55.97,80.60,69,0,5.82,BR
343,343,Kampot,10.62,104.18,87.80,52,0,5.82,KH
370,370,Khipro,25.83,69.38,84.58,19,0,7.74,PK
409,409,Filingué,14.35,3.32,75.56,23,0,11.56,NE
529,529,Sikandra Rao,27.70,78.40,83.26,22,0,8.12,IN


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
hotel_df= Ideal_criteria_df[["City", "Country", "Latitude", "Longitude"]].copy()
hotel_df["Hotel"]= ""
hotel_df.reindex()
hotel_df

,City,Country,Latitude,Longitude,Hotel
28,Mackay,AU,-21.15,149.20,
59,Ankazoabo,MG,-22.28,44.52,
66,Ingrāj Bāzār,IN,25.00,88.15,
180,Shima,CN,24.45,117.81,
222,Kareli,IN,22.92,79.07,
335,Barão de Melgaço,BR,-16.19,-55.97,
343,Kampot,KH,10.62,104.18,
370,Khipro,PK,25.83,69.38,
409,Filingué,NE,14.35,3.32,
529,Sikandra Rao,IN,27.70,78.40,


In [37]:


target_type = "lodging"
radius = 5000

# rewrite params dict

params = {
    "types": target_type,
    "radius": radius,
    "key": g_key
}

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():

    # get hotel info from df
    lat = row['Latitude']
    long = row['Longitude']

    # add keyword to params dict
    params['location'] = f"{lat},{long}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {'City'}.")
    response = requests.get(base_url, params=params).json()
    #print('\n', 'response is of datatype ', type(response), '\n')
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

hotel_df


Retrieving Results for Index 28: City.
Closest hotel is International Lodge Motel.
------------
Retrieving Results for Index 59: City.
Missing field/result... skipping.
------------
Retrieving Results for Index 66: City.
Closest hotel is Hotel Samrat Malda.
------------
Retrieving Results for Index 180: City.
Closest hotel is 龙海钻石大酒店.
------------
Retrieving Results for Index 222: City.
Closest hotel is बस्ती तालाब Pond.
------------
Retrieving Results for Index 335: City.
Closest hotel is Tuiuiu Pantanal Hotel.
------------
Retrieving Results for Index 343: City.
Closest hotel is Champa Lodge.
------------
Retrieving Results for Index 370: City.
Closest hotel is Raja Hotel.
------------
Retrieving Results for Index 409: City.
Missing field/result... skipping.
------------
Retrieving Results for Index 529: City.
Closest hotel is Murli Krishna Mangal Dham.
------------
Retrieving Results for Index 545: City.
Closest hotel is Casa Dora, Bubaque.
------------


,City,Country,Latitude,Longitude,Hotel,Hotel Name
28,Mackay,AU,-21.15,149.20,,International Lodge Motel
59,Ankazoabo,MG,-22.28,44.52,,NaN
66,Ingrāj Bāzār,IN,25.00,88.15,,Hotel Samrat Malda
180,Shima,CN,24.45,117.81,,龙海钻石大酒店
222,Kareli,IN,22.92,79.07,,बस्ती तालाब Pond
335,Barão de Melgaço,BR,-16.19,-55.97,,Tuiuiu Pantanal Hotel
343,Kampot,KH,10.62,104.18,,Champa Lodge
370,Khipro,PK,25.83,69.38,,Raja Hotel
409,Filingué,NE,14.35,3.32,,NaN
529,Sikandra Rao,IN,27.70,78.40,,Murli Krishna Mangal Dham


In [38]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
marker_locations = hotel_df[["Latitude", "Longitude"]]


In [39]:
# Add marker layer ontop of heat map
gmaps.configure(api_key=g_key)

fig= gmaps.figure(center=[0,0], zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, dissipating=False, max_intensity=100,
                                point_radius=5)

fig.add_layer(heat_layer)


# Create a marker_layer using the hotel name list to fill the info box

markers = gmaps.marker_layer(marker_locations,hotel_info)
fig.add_layer(markers)

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))